### PPAML Challenge Problem 7

University of California, Berkeley

Submission for BLOG by Prof. Stuart Russell's group.

#### Flu spread model

<img src="images/gmrf.png" />

<img src="images/adjacency.png" />

<img src="images/model.png" />

In [1]:
import functools
import json
import numpy as np
import pandas as pd
import re

from collections import defaultdict

In [2]:
TRAINING_SIZE = 'Small'

#### Load the data.

Need to make sure to load from the right training data size.

In [3]:
ili_data            = pd.read_csv("data/%s/input/Flu_ILI.csv" % TRAINING_SIZE)
tweets_data         = json.load(open("data/%s/input/Flu_Vacc_Tweet_TRAIN.json" % TRAINING_SIZE))
states              = json.load(open("data/%s/input/StateInfo.json" % TRAINING_SIZE))
regions_to_counties = json.load(open("data/%s/input/Region2CountyMap.json" % TRAINING_SIZE))
county_adjacency    = json.load(open("data/%s/input/county_adjacency_lower48.json" % TRAINING_SIZE))

#### List the dates.

It's important that the dates are in chronological order.<br/>
The index of the event is important for writing observations.

In [4]:
dates = ['08/10/2013', '08/17/2013', '08/24/2013', '08/31/2013', '09/07/2013', '09/14/2013', '09/21/2013',
         '09/28/2013', '10/05/2013', '10/12/2013', '10/19/2013', '10/26/2013', '11/02/2013', '11/09/2013',
         '11/16/2013', '11/23/2013', '11/30/2013', '12/07/2013', '12/14/2013', '12/21/2013', '12/28/2013',
         '01/04/2014', '01/11/2014', '01/18/2014', '01/25/2014', '02/01/2014', '02/08/2014', '02/15/2014',
         '02/22/2014', '03/01/2014', '03/08/2014', '03/15/2014', '03/22/2014', '03/29/2014', '04/05/2014',
         '04/12/2014', '04/19/2014', '04/26/2014', '05/03/2014', '05/10/2014', '05/17/2014', '05/24/2014',
         '05/31/2014', '06/07/2014', '06/14/2014', '06/21/2014', '06/28/2014', '07/05/2014', '07/12/2014',
         '07/19/2014', '07/26/2014', '08/02/2014']

In [5]:
print("Number of dates:", len(dates))

Number of dates: 52


#### Compute county statistics.

We need to compute *covariates* and *population* for each county.

$$\begin{align}
    N_c & = \texttt{ loaded from data }\\
    X_{c,t} & = \begin{bmatrix} 
                    \log{(\frac{S_{c,t} + \epsilon_2}{\tilde{N}_c})} & 
                    \log{(\frac{V_{c,t} + \epsilon_3}{1-V_{c,t}+\epsilon_3})} 
                \end{bmatrix}^T\\
\end{align}$$

Where 
$$\begin{align}
    \epsilon_2 & = 0.1\\
    \epsilon_3 & = 0.001\\
\end{align}$$

The covariate matrices should be of size $n$ by $d$.<br />
The population vector should be of size $n$ by $1$.

In [6]:
fips_to_cov1 = defaultdict(list)
fips_to_cov2 = defaultdict(list)
fips_to_pop = {}

In [7]:
for fips_code, blob in tweets_data.items():
    
    if not 'Vaccination percentage %' in blob.keys():
        continue
        
    for date in dates:
    
        cov1 = np.log((blob['No. of Tweets'][date] + 0.1) / blob['Population, 2014 estimate'] * 1000) 
        cov2 = np.log((blob['Vaccination percentage %'][date] / 100 + 0.001) / 
                      (1-blob['Vaccination percentage %'][date] / 100 + 0.001))
        
        fips_to_cov1[fips_code].append(cov1)
        fips_to_cov2[fips_code].append(cov2)
        
    fips_to_pop[fips_code] = blob['Population, 2014 estimate']

#### Construct sets of regions and counties.

We extract regions and counties for *only* the relevant counties from the training data.<br />

In [8]:
regions = set()
for i, col in enumerate(ili_data.columns):
    if i > 3:
        regions.add(col)

In [9]:
counties = set()
for r in regions:
    counties = counties.union(set(regions_to_counties[r].keys()))

In [10]:
print('Number of regions:', len(regions))
print('Number of counties:', len(counties))

Number of regions: 9
Number of counties: 82


#### Save county data.

Note: we assign an index to each county (somewhat arbitrarily).

We also create (and make sure to use) the following dictionaries:
- index_to_county
- county_to_index

In [11]:
county_to_index = {}
for i, fips in enumerate(counties):
    county_to_index[fips] = i
index_to_county = {v: k for k, v in county_to_index.items()}

In [12]:
county_pop_matrix = []
cov1_matrix = []
cov2_matrix = []

for i, fips in index_to_county.items():
    county_pop_matrix.append(fips_to_pop[fips])
    cov1_matrix.append(fips_to_cov1[fips])
    cov2_matrix.append(fips_to_cov2[fips])

county_pop_matrix = np.array(county_pop_matrix)
cov1_matrix = np.array(cov1_matrix)
cov2_matrix = np.array(cov2_matrix)
    
np.savetxt('data_processed/county_pops.txt', county_pop_matrix)
np.savetxt('data_processed/covariates1.txt', cov1_matrix)
np.savetxt('data_processed/covariates2.txt', cov2_matrix)

In [13]:
print(cov1_matrix.shape)
print(cov2_matrix.shape)
print(county_pop_matrix.shape)

(82, 52)
(82, 52)
(82,)


#### Map regions to counties.

We construct a resulting matrix $A$ that contains
$$A_{i,j} = \begin{cases} 1 & \mbox{if region } i \mbox{ contains county } j\\
                          0 & \mbox{otherwise}  \end{cases}$$
                                               
Also calculate the region population by
$$N_r = \sum_{c \in r} N_c$$

The resulting matrix should be of size $m$ by $n$.

In [14]:
region_to_index = {}
for i, r in enumerate(regions):
    region_to_index[r] = i
index_to_region = {v: k for k, v in region_to_index.items()}

In [15]:
county_map_matrix = np.zeros((len(regions), len(counties)))
region_pop_matrix = [0] * len(regions)

for i, r in index_to_region.items():
    
    for fips in regions_to_counties[r]:
        if fips not in county_to_index:
            continue
        county_map_matrix[i][county_to_index[fips]] = 1
        region_pop_matrix[i] += fips_to_pop[fips]

In [16]:
np.savetxt('data_processed/county_map.txt', county_map_matrix)
np.savetxt('data_processed/region_pops.txt', region_pop_matrix)

In [17]:
print(county_map_matrix.shape)

(9, 82)


#### Construct correlation covariance matrix.

Assuming the time series model, we have a single covariance matrix of size $n$ by $n$ (where $n$ is the number of counties). Construct following precision matrix with hyperparameter $\tau_1 \sim \mbox{Gamma}(3, 0.1)$.
$$\Sigma^{-1} = \tau_1 (D_w - W)$$

Therefore the output from this step is a matrix
$$\Sigma^{-1} = (D_w - W) + 0.01 I$$

Where $W$ is a symmetric matrix:
$$W_{i,j} = \begin{cases} 1 & \mbox{if } i \mbox{ neighbors } j\\ 0 & \mbox{otherwise} \end{cases}$$
And $D_w$ is a diagonal matrix:
$$Dw_{i,i} = \sum_j W_{i,j}$$
And $I$ is meant for regularization to ensure the matrix is positive definite.

In [18]:
W = np.zeros(((len(counties), len(counties))))

for blob in county_adjacency.values():
    
    fips = blob[1]
    neighbors = blob[2].values()
    
    if fips not in county_to_index:
        continue
        
    i = county_to_index[fips]
    for n in neighbors:
        if not n in county_to_index:
            continue
        j = county_to_index[n]
        if i == j:
            continue
        W[i][j] = 1
        W[j][i] = 1

In [23]:
np.savetxt("data_processed/W.txt", W)

In [24]:
D = np.zeros(((len(counties), len(counties))))

for i in range(len(counties)):
    D[i,i] = np.sum(W[i,:])

In [31]:
print(D)

[[ 5.  0.  0. ...,  0.  0.  0.]
 [ 0.  7.  0. ...,  0.  0.  0.]
 [ 0.  0.  5. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  4.  0.  0.]
 [ 0.  0.  0. ...,  0.  6.  0.]
 [ 0.  0.  0. ...,  0.  0.  5.]]


In [32]:
print(np.diag(D))

[ 5.  7.  5.  7.  9.  3.  4.  6.  5.  7.  7.  7.  5.  6.  6.  6.  4.  5.
  4.  6.  8.  6.  5.  7.  6.  5.  4.  3.  3.  3.  4.  6.  3.  6.  5.  7.
  6.  5.  6.  5.  5.  7.  5.  5.  4.  6.  6.  3.  3.  7.  6.  6.  5.  7.
  7.  4.  7.  4.  5.  3.  3.  6.  6.  7.  7.  4.  3.  5.  4.  4.  5.  5.
  8.  5.  3.  4.  4.  6.  6.  4.  6.  5.]


In [30]:
np.savetxt('data_processed/D.txt', np.diag(D))
# np.savetxt('data_processed/corr_cov.txt', eye(len(counties)))

#### Write observations.

We ignore any entries that are NaN in the training data.

In [43]:
obs_file = open('flu_spread_obs.blog', 'w')

In [44]:
for i, row in ili_data.iterrows():
    
    for j, region in index_to_region.items():

        if pd.isnull(row[region]):
            continue
            
        rate = float(row[region].strip('%')) / 100
        obs_file.write('obs region_rate(regions[%d], weeks[%d]) = %0.4f;\n' % (j, i, rate))

#### Write queries.

Query for every county at every timestep.

In [45]:
queries_file = open('flu_spread_queries.blog', 'w')

In [46]:
for j, _ in enumerate(dates):
    for i, county in index_to_county.items():        
        queries_file.write('query county_rate(counties[%d], weeks[%d]);\n' % (i, j))

#### Compile the model and run it.

In [46]:
!make

cat flu_spread_model.blog flu_spread_obs.blog flu_spread_queries.blog > \
		bin/flu_spread_compiled.blog;
/Users/tony/Projects/swift/swift \
		-i bin/flu_spread_compiled.blog \
		-o bin/flu_spread_compiled_liuwest.cpp \
		-e ParticleFilter --particle 10000;
correctly translated model file <bin/flu_spread_compiled.blog>!
awk '/perturb/ {$0="//"$0}1' bin/flu_spread_compiled_liuwest.cpp \
        > bin/flu_spread_tmp.cpp;
awk '/int _cur_loop/ {$0="//"$0}1' bin/flu_spread_tmp.cpp \
		> bin/flu_spread_compiled.cpp;
rm bin/flu_spread_tmp.cpp
g++ -I bin -Ofast -std=c++11 \
		bin/flu_spread_compiled.cpp \
		/Users/tony/Projects/swift/src/random/*.cpp \
		-o flu_spread_compiled -larmadillo;
In file included from bin/flu_spread_compiled.cpp:50:
In file included from bin/util/Hist_matrix.h:14:
In file included from bin/util/util_matrix.h:26:
bin/util/util_MCMC.h:31:20: warning: delete called on 'swift::MCMCObject' that
      is abstract but has non-virtual destructor [-Wdelete-non-virtual-dtor]
 

In [47]:
!./flu_spread_compiled > out/output_small.txt

flu_spread_compiled(44007,0x7fff7c572000) malloc: *** error for object 0x7feada8b9800: pointer being freed was not allocated
*** set a breakpoint in malloc_error_break to debug
/bin/sh: line 1: 44007 Abort trap: 6           ./flu_spread_compiled > out/output_small.txt


#### Step 10. Post-processing.

In [48]:
output_file = open('out/output_small.txt')
lines = output_file.readlines()

In [53]:
index_and_timestamp_to_county_rate = defaultdict(dict)

In [71]:
m = re.search("\\d+", "0 adsfasdf@2")

In [72]:
m.group(0)

'0'